# TruLens for LLMs: Quickstart

In this quickstart you will create a simple LLM Chain and learn how to log it and get feedback on an LLM response.

Note: If you haven't already, make sure to set up your local .env file with your OpenAI and Huggingface Keys. Your .env file should be in the same directory that you run this notebook. If you need help, take a look at the [.env.example](https://github.com/truera/trulens_private/blob/e8b11c4689e644687d2eafe09d90d8d7774b581c/trulens_eval/trulens_eval/.env.example#L4)

## Import from LangChain and TruLens

In [1]:
from IPython.display import JSON

# imports from trulens to log and get feedback on chain
from trulens_eval.keys import *
from trulens_eval import TruChain, Feedback, OpenAI, Huggingface, Tru

# imports from langchain to build app
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate, PromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate

KEY SET: OPENAI_API_KEY
KEY SET: PINECONE_API_KEY
KEY SET: PINECONE_ENV
KEY SET: HUGGINGFACE_API_KEY
KEY SET: SLACK_TOKEN
KEY SET: SLACK_SIGNING_SECRET
KEY SET: COHERE_API_KEY


## Create Simple LLM Application

This example uses a LangChain framework and OpenAI LLM

In [2]:
full_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template=
        "Provide a helpful response with relevant background information for the following: {prompt}",
        input_variables=["prompt"],
    )
)

chat_prompt_template = ChatPromptTemplate.from_messages([full_prompt])

chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.9)

chain = LLMChain(llm=chat, prompt=chat_prompt_template, verbose=True)

## Send your first request to your new app, asking how to adopt a dog in spanish

In [3]:
prompt_input = 'Como adopto un perro?'

In [4]:
gpt3_response = chain(prompt_input)

display(gpt3_response)



> Entering new LLMChain chain...
Prompt after formatting:
Human: Provide a helpful response with relevant background information for the following: Como adopto un perro?

> Finished chain.


{'prompt': 'Como adopto un perro?',
 'text': 'Adoptar un perro es una gran responsabilidad, pero también puede ser una experiencia muy gratificante. Primero, debes investigar y encontrar un refugio para animales cercano a ti. A menudo, estos refugios tienen perros que necesitan hogares amorosos. También puedes buscar en línea en sitios web como Adopt-a-Pet.com o Petfinder.com. \n\nCuando encuentres un perro que te guste, asegúrate de conocer su historial médico y comportamental. Pregunta sobre su peso, edad, si está vacunado, si tiene alguna enfermedad o necesidades especiales, y si ha tenido algún comportamiento agresivo. Los refugios generalmente hacen una evaluación de comportamiento, pero también es importante que puedas observar al perro con tus propios ojos.\n\nDespués de seleccionar a tu perro, deberás llenar una solicitud para la adopción y pagar una tarifa. Esta tarifa puede variar dependiendo de la organización, pero a menudo incluye el costo de la esterilización o castración

# Instrument chain for logging with TruLens

In [5]:
truchain: TruChain = TruChain(chain, chain_id='Chain1_ChatApplication')

In [6]:
# Instrumented chain can operate like the original:

gpt3_response = truchain(prompt_input)

display(gpt3_response)



> Entering new LLMChain chain...
Prompt after formatting:
Human: Provide a helpful response with relevant background information for the following: Como adopto un perro?

> Finished chain.


{'prompt': 'Como adopto un perro?',
 'text': 'Adoptar un perro puede ser un proceso emocionante y satisfactorio, ya que estarás brindando un hogar amoroso a una mascota necesitada. Hay varias opciones para adoptar un perro, como ir a un refugio local de animales o trabajar con una organización de rescate de animales.\n\nAntes de adoptar un perro, es importante evaluar tu estilo de vida y tu capacidad para cuidar de una mascota. Las necesidades de los perros varían según la edad, la raza y la personalidad, y es importante encontrar un animal que se adapte bien a tu estilo de vida y pueda recibir la atención y el cuidado adecuados.\n\nUna vez que hayas encontrado un perro que te guste, es probable que debas completar una solicitud de adopción y someterte a una entrevista con un representante de la organización. También es posible que debas pagar una tarifa de adopción y proporcionar información sobre tus antecedentes y situación de vivienda.\n\nAdoptar un perro puede ser una experiencia 

In [7]:
# But can also produce a log or "record" of the execution of the chain:

gpt3_response, record = truchain.call_with_record(prompt_input)

JSON(record)



> Entering new LLMChain chain...
Prompt after formatting:
Human: Provide a helpful response with relevant background information for the following: Como adopto un perro?

> Finished chain.


<IPython.core.display.JSON object>

In [8]:
# We can log the records but first we need to log the chain itself:

tru = Tru()

tru.add_chain(chain_json=truchain.json)

*** Creating new Tru singleton instance for name = None ***


In [9]:
# Now the record:

tru.add_record(
    prompt=prompt_input, # prompt input
    response=gpt3_response['text'], # LLM response
    record_json=record # record is returned by the TruChain wrapper
)

# Note that the `add_record` call automatically sets the `record_id` field of the
# `record_json` to the returned record id. Retrieving it from the output of `add_record` is not 
# necessary.

'record_hash_7c9388a886ec7489abd77ae3d12b5661'

In [10]:
# Initialize Huggingface-based feedback function collection class:
hugs = Huggingface()

# Define a language match feedback function using HuggingFace.
f_lang_match = Feedback(hugs.language_match).on(
    text1="prompt", text2="response"
)

*** Creating new Endpoint singleton instance for name = huggingface ***
*** Creating huggingface endpoint ***


huggingface api: 0requests [00:00, ?requests/s]

*** Creating new TP singleton instance for name = None ***


In [11]:
# This might take a moment if the public api needs to load the language model
# used in the feedback function:
feedback_result = f_lang_match.run_on_record(
    chain_json=truchain.json, record_json=record
)

In [12]:
JSON(feedback_result)

<IPython.core.display.JSON object>

In [13]:
# Alternatively, run a collection of feedback functions:

feedback_results = tru.run_feedback_functions(
    record_json=record,
    feedback_functions=[f_lang_match]
)

In [14]:
display(feedback_results)

[{'_success': True,
  'feedback_id': 'feedback_hash_26c84c76d907d808c36028e024af63c0',
  'record_id': 'record_hash_7c9388a886ec7489abd77ae3d12b5661',
  'language_match': 0.005009680444345577}]

In [15]:
# These can be logged:

tru.add_feedbacks(feedback_results)

TypeError: Tru.add_feedbacks() missing 1 required positional argument: 'db'

## Run the TruLens dashboard to explore the quality of your LLM chain

In [ ]:
tru.run_dashboard() # open a local streamlit app to explore

## Automatic Logging

The above logging and feedback function evaluation steps can be done by TruChain.

In [16]:
truchain: TruChain = TruChain(
    chain,
    chain_id='Chain1_ChatApplication',
    feedbacks=[f_lang_match],
    tru=tru
)

# Note: providing `db: TruDB` causes the above constructor to log the wrapped chain in the database specified.

# Note: any `feedbacks` specified here will be evaluated and logged whenever the chain is used.

✅ Chain1_ChatApplication -> SQLite(default.sqlite)
✅ feedback_hash_26c84c76d907d808c36028e024af63c0 -> SQLite(default.sqlite)


In [17]:
truchain("This will be automatically logged.")



> Entering new LLMChain chain...
Prompt after formatting:
Human: Provide a helpful response with relevant background information for the following: This will be automatically logged.

> Finished chain.


{'prompt': 'This will be automatically logged.',
 'text': 'Automatic logging is a feature used in many different types of technology, such as computer programs, web applications, and electronic devices. It involves capturing and recording specific information about user behavior, system events, and other important data. This information is stored in a log file, which can be used for troubleshooting, auditing, monitoring, or analysis purposes. By automatically logging certain activities, organizations can ensure compliance with regulations, maintain security and integrity of their systems, and make informed decisions based on data insights. So when you see the message "This will be automatically logged", it means that the system is capturing specific information required for tracking and analysis.'}

Running feedback functions now.


## Out-of-band Feedback evaluation

In the above example, the feedback function evaluation is done in the same process as the chain evaluation. The alternative approach is the use the provided persistent evaluator started via `tru.start_deferred_feedback_evaluator`. Then specify the `feedback_mode` for `TruChain` as `deferred` to let the evaluator handle the feedback functions.

For demonstration purposes, we start the evaluator here but it can be started in another process.

In [18]:
truchain: TruChain = TruChain(
    chain,
    chain_id='Chain1_ChatApplication',
    feedbacks=[f_lang_match],
    tru=tru,
    feedback_mode="deferred"
)

✅ Chain1_ChatApplication -> SQLite(default.sqlite)
✅ feedback_hash_26c84c76d907d808c36028e024af63c0 -> SQLite(default.sqlite)


In [ ]:
tru.start_deferred_feedback_evaluator()

In [19]:
truchain("This will be logged by deferred evaluator.")



> Entering new LLMChain chain...
Prompt after formatting:
Human: Provide a helpful response with relevant background information for the following: This will be logged by deferred evaluator.

> Finished chain.


{'prompt': 'This will be logged by deferred evaluator.',
 'text': 'Deferred evaluation is a programming technique where a value is not immediately computed when called, but is instead evaluated at a later time or only when needed. This approach can improve performance by only computing values when necessary, rather than wasting resources on values that may never be used. \n\nIn the context of logging, a deferred evaluator is a tool that allows for the delayed evaluation of log entries. This can be useful in situations where logging is performed continuously, but the actual processing and analysis of the logs happens on a separate schedule. By deferring the evaluation of log entries until they are needed, a deferred evaluator can help reduce the overhead associated with logging by only computing values when they are actually needed.\n\nOverall, the use of a deferred evaluator for logging provides a flexible and efficient way to manage logging data, allowing for delayed evaluation and pr

Feedback functions evaluated in the deferred manner can be seen in the "Progress" page of the TruLens dashboard.